In [1]:
import pandas as pd
import os
from copy import deepcopy
import numpy as np
from math import isclose

os.chdir("..")
from adaptive_multigrid import give_data_individual_names, find_best_individual_name, build_individual_dict_from_param_grid, update_param_grid
from gvs_threshold_calc import calc_gvs_thresh_ineq_met
from model.config.params import *


fitness_func = calc_gvs_thresh_ineq_met
sweep = "gateway_viability_sweep_ag3_"
kpis = pd.read_csv("simulation_data/{}.csv".format(sweep), index_col=0)
exec("param_config = {}".format(sweep))
next_name = sweep[:-2] + str(int(sweep[-2]) + 1) + "_"

variable_params = ['session_token_bucket_coefficient', 'gateway_fee_per_relay', 'application_fee_per_relay', 'gateway_minimum_stake', 'minimum_application_stake']
control_params = ['application_max_number', 'relays_per_session_gamma_distribution_scale']

params_to_evolve = {}
for x in variable_params:
    params_to_evolve[x] = gateway_viability_sweep_ag2_[x]

params_for_setup_only = {}
for x in control_params:
    params_for_setup_only[x] = gateway_viability_sweep_ag2_[x]

initial_param_grid = params_to_evolve
other_params_to_sweep = params_for_setup_only




/Users/seanmcowen/opt/anaconda3/envs/BlockScience/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
new_param_grid = deepcopy(initial_param_grid)
new_best_name = ""
new_best_fitness = -np.inf
times_repeat_best_name = 0
times_repeat_best_fitness = 0

old_param_grid = deepcopy(new_param_grid)
old_best_name = new_best_name
old_best_fitness = new_best_fitness

individual_dict = build_individual_dict_from_param_grid(old_param_grid)
        
        

data = kpis
data = give_data_individual_names(data)

new_best_name, new_best_val = find_best_individual_name(
            data, fitness_func=fitness_func, name_biased_towards=old_best_name
        )

assert not (
            new_best_name is None
        ), "The best individual should be somewhere in this dictionary."
print(new_best_name)

new_best_individual = individual_dict[new_best_name]

if new_best_name == old_best_name:
    times_repeat_best_name = times_repeat_best_name + 1
else:
    times_repeat_best_name = 0

if isclose(new_best_fitness, old_best_fitness):
    times_repeat_best_fitness = times_repeat_best_fitness + 1
else:
    times_repeat_best_fitness = 0
    
new_param_grid = update_param_grid(
                old_param_grid=old_param_grid, best_individual=new_best_individual
            )

Building the individual dict.
Individual dict is {'stbc:25.0_gfpr:20.0_afpr:25.0_gms:100000000000.0_ams:150000000000.0': {'session_token_bucket_coefficient': 25.0, 'gateway_fee_per_relay': 20.0, 'application_fee_per_relay': 25.0, 'gateway_minimum_stake': 100000000000.0, 'minimum_application_stake': 150000000000.0}, 'stbc:25.0_gfpr:20.0_afpr:25.0_gms:100000000000.0_ams:200000000000.0': {'session_token_bucket_coefficient': 25.0, 'gateway_fee_per_relay': 20.0, 'application_fee_per_relay': 25.0, 'gateway_minimum_stake': 100000000000.0, 'minimum_application_stake': 200000000000.0}, 'stbc:25.0_gfpr:20.0_afpr:25.0_gms:150000000000.0_ams:150000000000.0': {'session_token_bucket_coefficient': 25.0, 'gateway_fee_per_relay': 20.0, 'application_fee_per_relay': 25.0, 'gateway_minimum_stake': 150000000000.0, 'minimum_application_stake': 150000000000.0}, 'stbc:25.0_gfpr:20.0_afpr:25.0_gms:150000000000.0_ams:200000000000.0': {'session_token_bucket_coefficient': 25.0, 'gateway_fee_per_relay': 20.0, 'app

We have added individual names for the various parameter combinations.
The DataFrame now looks like                                                     Net Minting Rate  \
individual_name                                                        
stbc:25.0_gfpr:20.0_afpr:25.0_gms:100000000000....          0.000007   
stbc:25.0_gfpr:20.0_afpr:25.0_gms:100000000000....          0.000006   
stbc:25.0_gfpr:20.0_afpr:25.0_gms:100000000000....          0.000007   
stbc:25.0_gfpr:20.0_afpr:25.0_gms:100000000000....          0.000006   
stbc:25.0_gfpr:20.0_afpr:25.0_gms:100000000000....          0.000007   

                                                       KPI C  \
individual_name                                                
stbc:25.0_gfpr:20.0_afpr:25.0_gms:100000000000....  0.173062   
stbc:25.0_gfpr:20.0_afpr:25.0_gms:100000000000....  0.178044   
stbc:25.0_gfpr:20.0_afpr:25.0_gms:100000000000....  0.223872   
stbc:25.0_gfpr:20.0_afpr:25.0_gms:100000000000....  0.198511   
stbc:25.0_g

In [6]:
new_param_grid

{'session_token_bucket_coefficient': [118.75, 212.5],
 'gateway_fee_per_relay': [20.0, 22.5],
 'application_fee_per_relay': [25.0, 27.5],
 'gateway_minimum_stake': [125000000000.0, 150000000000.0],
 'minimum_application_stake': [175000000000.0, 200000000000.0]}

In [30]:
from psuu import build_next_param_config_code

build_next_param_config_code(
    next_name, new_param_grid, control_params, param_config
)

Add the following to model/config/params.py:

gateway_viability_sweep_ag3_ = build_params("Base")
gateway_viability_sweep_ag3_["session_token_bucket_coefficient"] = [25.0, 118.75]
gateway_viability_sweep_ag3_["gateway_fee_per_relay"] = [22.5, 25.0]
gateway_viability_sweep_ag3_["application_fee_per_relay"] = [25.0, 27.5]
gateway_viability_sweep_ag3_["gateway_minimum_stake"] = [100000000000.0, 125000000000.0]
gateway_viability_sweep_ag3_["minimum_application_stake"] = [175000000000.0, 200000000000.0]
gateway_viability_sweep_ag3_["application_max_number"] = [5, 20, 100]
gateway_viability_sweep_ag3_["relays_per_session_gamma_distribution_scale"] = [100000, 300000, 900000]


In [21]:
for x in control_params:
    print(x)

application_max_number
relays_per_session_gamma_distribution_scale


In [26]:
param_config[x]

[100000, 300000, 900000]

In [17]:
str(new_param_grid["gateway_minimum_stake"])

'[100000000000.0, 125000000000.0]'

In [11]:

gateway_viability_sweep_ag3_["session_token_bucket_coefficient"] = [118.75, 212.5]
gateway_viability_sweep_ag3_["gateway_fee_per_relay"] = [20.0, 22.5]
gateway_viability_sweep_ag3_["application_fee_per_relay"] = [25.0, 27.5]
gateway_viability_sweep_ag3_["gateway_minimum_stake"] = [125000000000.0, 150000000000.0]
gateway_viability_sweep_ag3_["minimum_application_stake"] = [
    175000000000.0,
    200000000000.0,
]
gateway_viability_sweep_ag3_["application_max_number"] = [5, 20, 100]
gateway_viability_sweep_ag3_["relays_per_session_gamma_distribution_scale"] = [
    100000,
    300000,
    900000,
]

create_sweep(
    "gateway_viability_sweep_ag3_",
    gateway_viability_sweep_ag3_,
    config_option_map_sweep,
)

'gateway_viability_sweep_ag3_'

In [3]:
gateway_viability_sweep_ag2_

{'minimum_stake_servicer': [15000000000.0],
 'minimum_stake_period_servicer': [10],
 'minimum_pause_time': [10],
 'max_chains_servicer': [15],
 'relays_to_tokens_multiplier': [161.29],
 'slash_fraction_downtime': [1e-06],
 'downtime_jail_duration': [3600000000000],
 'minimum_servicers_per_session': [1],
 'maximum_servicers_per_session': [5],
 'application_fee_per_relay': [25.0, 30.0],
 'minimum_application_stake': [150000000000.0, 200000000000.0],
 'app_burn_per_session': [0],
 'app_burn_per_relay': [0],
 'block_proposer_allocation': [0.05],
 'dao_allocation': [0.1],
 'servicer_allocation': [0.85],
 'stake_per_app_delegation': [15000000000.0],
 'gateway_fee_per_relay': [20.0, 25.0],
 'gateway_minimum_stake': [100000000000.0, 150000000000.0],
 'session_token_bucket_coefficient': [25.0, 212.5],
 'dao_fee_percentage': [0.1],
 'validator_fee_percentage': [0.9],
 'transaction_fee': [0.01],
 'min_bootstrap_gateway_fee_per_relay': [8.5e-07],
 'maturity_relay_charge': [1.971e-06],
 'gateway_bo